# `Jai Chaudhry 2K18-SE-069`

## Experiment - 5

- Write a program to demonstrate the working of the decision tree based CART algorithm. 
- With the help of data set used in above experiment build the decision tree and classify a new sample. 
- Then Finding Accuracy, Precision, Recall and ROC_AUC Score of the Decision Tree

In [1]:
import numpy as np
import pandas as pd
eps = np.finfo(float).eps
from numpy import log2 as log

from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score


In [6]:
# Find Gini Index for a particular attribute
def find_gini_index(df,attribute,printy=False):
    Class = df.keys()[-1]  
    
    target_variables = df[Class].unique()
    variables = df[attribute].unique()
    gini = 0
    
    for variable in variables:
        den = len( df[attribute][df[attribute]==variable] )
        
        fraction2 = 1
        for target_var in target_variables:
            num = len(df[attribute][df[attribute]==variable][df[Class]==target_var])
            fraction = num / (den + eps)
            fraction2 -= fraction**2
        if printy:
            print(variable,": ",fraction2)
        fraction2 = fraction2 * (den/len(df))
        gini += fraction2 
        
    return gini


In [7]:
# Get The one that is the best divider for current node
def find_winner(df):
    Gini_att = []
    IG = []
    
    for key in df.keys()[:-1]:
        gini = find_gini_index(df,key)        
        IG.append(gini)

    return df.keys()[:-1][np.argmin(IG)]                                


## Get the subset of data that has a specific attr.'s value as x 
def get_subtable(df, node ,value): 
    return df[df[node] == value].reset_index(drop=True) 



In [8]:
# Main Driver Function to create Tree
def buildTree(df,TgtClass,previous_winner=0,tree=None): 
    Class = df.keys()[-1]   #To make the code generic, changing target variable class name
    
    # Here we build our decision tree
    
    # Get attribute with maximum information gain
    node = find_winner(df)
    
    print(node)
    
    # Create an empty dictionary to create tree    
    if tree is None:                    
        tree={}
        tree[node] = {}
    
    if previous_winner == node:
        clValue,counts = np.unique(df[TgtClass],return_counts = True) 
        tree[node][df[node][0]] = clValue[np.argmax(counts)]
        return tree
    
    # Get distinct value of that attribute e.g Salary is node and Low,Med and High are values
    attValue = np.unique(df[node])

   # We make loop to construct a tree by calling this function recursively. 
   # In this we check if the subset is pure and stops if it is pure. 
    
    for value in attValue:
        
        subtable = get_subtable(df,node,value)
        clValue,counts = np.unique(subtable[TgtClass],return_counts = True)   
        
        if len(counts) == 1:                                                   # Checking purity of subset
            tree[node][value] = clValue[0]   
        else:        
            tree[node][value] = buildTree(subtable,TgtClass,node)            # Calling the function recursively 
                   
    return tree


# Define the Dataset : (Sample Class Dataset)

In [9]:
outlook = 'sunny,sunny,overcast,rain,rain,rain,overcast,sunny,sunny,rain,sunny,overcast,overcast,rain'.split(',')
temp='hot,hot,hot,mild,cold,cold,cold,mild,cold,mild,mild,mild,hot,mild'.split(',')
humidity = 'high,high,high,high,normal,normal,normal,high,normal,normal,normal,high,normal,high'.split(',')
windy = 'weak,strong,weak,weak,weak,strong,strong,weak,weak,weak,strong,strong,weak,strong'.split(',')
play = 'no,no,yes,yes,yes,no,yes,no,yes,yes,yes,yes,yes,no'.split(',')

dataset=0

dataset = {'outlook':outlook,'temp':temp,'humidity':humidity,'windy':windy,'play':play}
df = pd.DataFrame(dataset,columns=['outlook','temp','humidity','windy','play'])

df.head()

,outlook,temp,humidity,windy,play
0,sunny,hot,high,weak,no
1,sunny,hot,high,strong,no
2,overcast,hot,high,weak,yes
3,rain,mild,high,weak,yes
4,rain,cold,normal,weak,yes


In [10]:
t = buildTree(df,'play')

import pprint      
pprint.pprint(t)   


outlook
windy
humidity
{'outlook': {'overcast': 'yes',
             'rain': {'windy': {'strong': 'no', 'weak': 'yes'}},
             'sunny': {'humidity': {'high': 'no', 'normal': 'yes'}}}}


# Visualizing Decision Tree

In [11]:
import pydot

dict = {}

def draw(parent_name, child_name):
    edge = pydot.Edge(parent_name, child_name)
    graph.add_edge(edge)


def visit(node, parent=None):
    for k,v in node.items():
        if isinstance(v, type(dict)):
            # We start with the root node whose parent is None
            # we don't want to graph the None node
            if parent:
                draw(parent, k)
            visit(v, k)
            
        else:
            draw(parent, k)
            # drawing the label using a distinct name
            draw(k, k+'_'+v)


graph = pydot.Dot(graph_type='graph')

visit(t)

graph.write_png('Play_Tennis_CART.png')

<img src="Play_Tennis_CART.png" >

# Now Implementing Decision Tree on Adult DataSet

In [22]:
import copy 
import pandas as pd

def get_processed_data():

    data = pd.read_csv('C:\\Users\\Jai\\Desktop\\adult.csv',na_values='?')

    num_cols = list(data.select_dtypes(include=["number"]).columns)
    cat_cols = list(data.select_dtypes(exclude=["number"]).columns)
    
    ## Drop NaN valued tuples
    data = data.dropna() 
    data.reset_index(drop=True,inplace=True)
    
    print(type(data))
    
    ndata = data.select_dtypes(include=['number']) 
    cdata = data.select_dtypes(exclude=['number']) 

    # Categorical to Numerical
    cols = ndata.columns
    b = np.array(ndata)
    a = np.array(ndata,dtype=object)
    print(a.shape)

    for i in range(0,len(cols)):

        col = cols[i]
        meann = np.round(ndata[col].mean())

        for j in range(0,len(ndata)):
            if b[j][i] >= meann:
                a[j][i] = ">=" + str(meann)
            else:
                a[j][i] = "<" + str(meann)


    newdata = pd.DataFrame(a).applymap(str)
    ndata = newdata
    ndata.columns = cols
    
    ### Merging the Transformed Numeric Data with Categorical Data
    ndata.reset_index(drop=True, inplace=True)
    cdata.reset_index(drop=True,inplace=True)
    frames = [ndata,cdata]
    result = pd.concat(frames,axis = 1)
    
    ### Final DataFrame after Processing
    data =  result
    
    return data
        

In [23]:
data = get_processed_data()

<class 'pandas.core.frame.DataFrame'>
(45222, 6)


In [27]:
test_size = 0.3
train_size = int((1 - test_size)* len(data))

df_train = data[:train_size]
df_test = data[train_size:]

df_test.reset_index(drop=True)
df_train.reset_index(drop=True)

print("Train_size",len(df_train))
print("Test_size",len(df_test))

Train_size 31655
Test_size 13567


In [28]:
df_train = df_train.applymap(str) 

In [29]:
tree = buildTree(df_train,'income')  


relationship
education
capital-loss
occupation
workclass
age
capital-gain
hours-per-week
fnlwgt
native-country
age
age
fnlwgt
race
age
age
fnlwgt
age
age
race
age
age
native-country
hours-per-week
fnlwgt
age
age
age
age
fnlwgt
age
age
age
age
hours-per-week
age
fnlwgt
age
age
workclass
fnlwgt
native-country
workclass
age
age
fnlwgt
capital-gain
age
age
workclass
fnlwgt
hours-per-week
workclass
race
native-country
age
age
age
age
capital-gain
age
age
native-country
capital-gain
workclass
workclass
race
hours-per-week
age
age
capital-gain
fnlwgt
hours-per-week
age
age
age
age
age
native-country
hours-per-week
age
age
age
age
age
occupation
hours-per-week
capital-gain
occupation
fnlwgt
workclass
native-country
age
age
workclass
age
native-country
age
capital-loss
race
hours-per-week
fnlwgt
age
age
fnlwgt
age
age
fnlwgt
hours-per-week
capital-loss
race
age
age
capital-loss
hours-per-week
race
age
age
hours-per-week
workclass
age
age
capital-loss
hours-per-week
age
fnlwgt
hours-per-week
fnl

hours-per-week
fnlwgt
fnlwgt
age
age
hours-per-week
fnlwgt
age
age
age
capital-loss
fnlwgt
hours-per-week
age
age
age
age
hours-per-week
age
age
fnlwgt
workclass
native-country
hours-per-week
race
age
fnlwgt
fnlwgt
age
age
native-country
race
hours-per-week
fnlwgt
age
capital-gain
capital-gain
age
age
age
age
age
capital-loss
age
age
native-country
workclass
capital-gain
age
fnlwgt
race
age
age
hours-per-week
age
age
native-country
age
age
hours-per-week
capital-loss
age
age
workclass
workclass
fnlwgt
hours-per-week
hours-per-week
age
age
capital-gain
capital-loss
workclass
age
capital-gain
fnlwgt
race
age
age
fnlwgt
capital-loss
hours-per-week
age
age
capital-gain
age
hours-per-week
race
fnlwgt
age
age
capital-loss
age
age
fnlwgt
age
age
capital-loss
age
age
race
fnlwgt
age
age
capital-loss
fnlwgt
hours-per-week
age
age
age
age
hours-per-week
age
age
age
age
fnlwgt
age
age
fnlwgt
age
hours-per-week
age
age
capital-gain
capital-loss
race
fnlwgt
age
age
age
hours-per-week
fnlwgt
age
age

capital-loss
fnlwgt
capital-gain
age
age
hours-per-week
age
age
age
age
capital-gain
hours-per-week
age
age
age
age
age
native-country
race
fnlwgt
hours-per-week
capital-gain
fnlwgt
age
age
age
age
age
fnlwgt
fnlwgt
age
age
race
age
hours-per-week
age
age
capital-gain
hours-per-week
fnlwgt
age
age
age
age
age
fnlwgt
native-country
fnlwgt
age
age
age
workclass
capital-gain
capital-loss
age
race
age
age
gender
hours-per-week
fnlwgt
age
age
age
age
fnlwgt
age
age
age
age
race
fnlwgt
hours-per-week
age
age
hours-per-week
fnlwgt
age
age
age
age
fnlwgt
age
age
age
age
age
fnlwgt
hours-per-week
hours-per-week
age
age
fnlwgt
hours-per-week
age
age
age
age
hours-per-week
age
fnlwgt
age
age
fnlwgt
age
age
age
age
fnlwgt
age
age
age
age
age
capital-gain
race
fnlwgt
hours-per-week
capital-loss
age
age
age
hours-per-week
capital-loss
age
age
age
age
capital-loss
age
age
hours-per-week
age
fnlwgt
capital-loss
age
age
race
age
age
race
capital-loss
age
capital-gain
fnlwgt
age
age
age
age
fnlwgt
age
a

age
age
age
workclass
age
fnlwgt
age
age
hours-per-week
age
age
capital-gain
race
capital-loss
age
native-country
hours-per-week
fnlwgt
age
age
age
age
fnlwgt
age
age
age
age
fnlwgt
hours-per-week
age
age
age
age
hours-per-week
age
age
age
age
race
age
fnlwgt
fnlwgt
fnlwgt
age
age
fnlwgt
capital-loss
age
race
age
age
capital-gain
capital-loss
hours-per-week
race
fnlwgt
age
age
fnlwgt
workclass
age
age
age
age
age
age
age
native-country
workclass
age
age
age
age
age
age
age
workclass
age
race
native-country
fnlwgt
fnlwgt
age
age
age
age
race
fnlwgt
native-country
age
age
fnlwgt
age
age
age
age
fnlwgt
age
age
age
age
age
race
fnlwgt
age
age
age
age
age
age
age
fnlwgt
workclass
age
age
hours-per-week
fnlwgt
workclass
hours-per-week
age
age
fnlwgt
workclass
hours-per-week
age
age
age
age
age
age
age
hours-per-week
age
age
hours-per-week
workclass
age
age
age
native-country
workclass
race
fnlwgt
age
hours-per-week
hours-per-week
age
age
race
age
capital-gain
age
hours-per-week
fnlwgt
capita

workclass
native-country
race
gender
gender
race
fnlwgt
race
hours-per-week
age
marital-status
fnlwgt
marital-status
workclass
fnlwgt
age
marital-status
capital-gain
hours-per-week
native-country
age
fnlwgt
marital-status
occupation
age
gender
age
age
workclass
workclass
gender
race
gender
fnlwgt
capital-loss
gender
age
age
age
age
gender
age
age
gender
fnlwgt
age
workclass
gender
age
occupation
age
age
occupation
age
gender
fnlwgt
capital-loss
fnlwgt
age
age
race
gender
age
capital-loss
age
age
age
fnlwgt
age
age
age
age
age
age
occupation
fnlwgt
age
age
occupation
capital-loss
workclass
race
workclass
occupation
age
age
fnlwgt
age
age
occupation
workclass
race
fnlwgt
race
age
native-country
workclass
fnlwgt
gender
marital-status
age
age
age
age
marital-status
age
age
age
age
gender
capital-loss
age
age
marital-status
age
age
marital-status
age
race
age
age
age
age
fnlwgt
gender
age
marital-status
age
age
native-country
workclass
race
fnlwgt
capital-loss
gender
age
marital-status
age


occupation
hours-per-week
race
fnlwgt
hours-per-week
marital-status
fnlwgt
age
age
hours-per-week
marital-status
occupation
gender
fnlwgt
capital-loss
hours-per-week
age
age
age
capital-loss
fnlwgt
marital-status
native-country
age
marital-status
age
age
fnlwgt
gender
native-country
marital-status
race
gender
occupation
hours-per-week
workclass
age
workclass
capital-gain
occupation
marital-status
fnlwgt
gender
occupation
marital-status
fnlwgt
age
race
workclass
age
age
capital-gain
hours-per-week
age
workclass
fnlwgt
hours-per-week
marital-status
workclass
hours-per-week
fnlwgt
age
native-country
age
age
capital-gain
capital-loss
age
occupation
hours-per-week
fnlwgt
marital-status
marital-status
workclass
occupation
capital-gain
marital-status
race
fnlwgt
age
age
age
age
hours-per-week
hours-per-week
fnlwgt
marital-status
fnlwgt
age
age
marital-status
occupation
education
workclass
age
fnlwgt
occupation
age
occupation
fnlwgt
age
native-country
age
native-country
race
occupation
fnlwgt


# Performance Evaluation

In [30]:
limit = 2*14

def predict(tree,prev,sample,cnt):
    
    # Find the current attr / parameter
    # Check the value of that in sample
    # Then navigate through the tree accordingly
    
#     global err, exception, returning
    
    if cnt > limit:
#         err += 1
        return "idk"
    try:
        tkeys = list(tree.keys())
    except:
        return tree
    
    try:
        if len(tkeys)==1:
            # Found a category
            t=1
            return predict(tree[tkeys[0]],tkeys[0],sample,cnt+1)
        else:
            return predict(tree[sample[prev]],"-1",sample,cnt+1)
    except:
#         exception += 1
        return "idk"
    
#     returning += 1
    return "idk"


In [31]:
def print_metrics(preds,actual):
    
    # first converting to 0 / 1
    predictions = np.zeros(len(preds))
    y_actual = np.zeros(len(actual))
    
    print(len(actual))
    
    for i in range(0,len(actual)):
        if preds[i]=='>50K':
            predictions[i] = 1
            
        if actual[i]=='>50K':
            y_actual[i] = 1
    
#     print(y_actual)
#     print(predictions)
    
    # accuracy: (tp + tn) / (p + n)
    accuracy = accuracy_score(y_actual,predictions)*100
    print('Accuracy      : %f' % (accuracy),"%")

    # precision tp / (tp + fp)
    precision = precision_score(y_actual,predictions)*100
    print('Precision     : %f' % precision,"%")

    # recall: tp / (tp + fn)
    recall = recall_score(y_actual,predictions)*100
    print('Recall        : %f' % recall,"%")

    # f1: 2 tp / (2 tp + fp + fn)
    f1 = f1_score(y_actual,predictions)*100
    print('F1 score      : %f' % f1,"%")

    # ROC AUC
    auc = roc_auc_score(y_actual,predictions)*100
    print("roc_auc_score :",auc,"%")
    

In [33]:
y_actual = np.array(df_test.iloc[:,-1])
preds = []

for i in range(0,len(df_test)):
    try:
        preds.append(predict(tree,"-1",df_test.iloc[i].to_dict(),0))
    except:
        preds.append("idk")


In [34]:
print_metrics(preds,y_actual)

13567
Accuracy      : 80.762143 %
Precision     : 64.656433 %
Recall        : 51.846424 %
F1 score      : 57.547170 %
roc_auc_score : 71.16201081363138 %
